In [4]:
# Import necessary libraries
import pandas as pd
import pickle
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import numpy as np
import shap
from sentence_transformers import SentenceTransformer  # Optional: more specific

# Load preprocessed data
resume_data = pd.read_pickle('env/preprocessed_resumes.pkl')
with open('env/preprocessed_job_description.pkl', 'rb') as f:
    preprocessed_job_description = pickle.load(f)

# Load advanced language model SentenceTransformer
# Option 1
nlp_transformer = SentenceTransformer('all-MiniLM-L6-v2')
# Option 2
# nlp_transformer = pipeline('feature-extraction', model='sentence-transformers/all-MiniLM-L6-v2')

nlp_spacy = spacy.load('en_core_web_sm')

# Function for NER using spaCy
def extract_entities(text):
    doc = nlp_spacy(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Extract entities from resumes
resume_data['Entities'] = resume_data['Preprocessed Text'].apply(extract_entities)

# Function to compute sentence embeddings
def get_sentence_embedding(text):
    embedding = nlp_transformer.encode([text])  # Using encode method directly (if using SentenceTransformer)
    return np.mean(embedding, axis=0)  # Averaging the embeddings across sentences

# Compute embeddings
resume_embeddings = resume_data['Preprocessed Text'].apply(get_sentence_embedding)
job_description_embedding = get_sentence_embedding(preprocessed_job_description)

# Calculate cosine similarity
similarity_scores = []
for embedding in resume_embeddings:
    score = cosine_similarity([embedding], [job_description_embedding])[0][0]  # Avoid embedding[0] for better robustness
    similarity_scores.append(score)

resume_data['Similarity Score'] = similarity_scores

# Map job description keywords to resume content
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([preprocessed_job_description] + resume_data['Preprocessed Text'].tolist())
feature_names = vectorizer.get_feature_names_out()

job_tfidf = tfidf_matrix[0]
resume_tfidf = tfidf_matrix[1:]

keyword_matches = []
for i, resume_vector in enumerate(resume_tfidf):
    jd_keywords = set([feature_names[index] for index in job_tfidf.nonzero()[1]])
    resume_keywords = set([feature_names[index] for index in resume_vector.nonzero()[1]])
    common_keywords = jd_keywords.intersection(resume_keywords)
    keyword_matches.append(common_keywords)

resume_data['Keyword Matches'] = keyword_matches

# Save modeling results
resume_data.to_pickle('env/modeling_results.pkl')

print("Modeling completed successfully.")


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


Modeling completed successfully.
